In [87]:
from openfisca_survey_manager.scenarios import AbstractSurveyScenario
from openfisca_france import CountryTaxBenefitSystem
from openfisca_france.model.base import Famille, FoyerFiscal, Menage
import pandas as pd


In [88]:
class StrasbourgSurveyScenario(AbstractSurveyScenario):
    def __init__(self, data = None):
        super(StrasbourgSurveyScenario, self).__init__()

        tax_benefit_system = CountryTaxBenefitSystem()
        tax_benefit_system.load_extension('openfisca_france_local')

        input_data_frame = data.get('input_data_frame')
        self.used_as_input_variables = list(
            set(tax_benefit_system.variables.keys()).intersection(
                set(input_data_frame.columns)
                ))

        self.set_tax_benefit_systems(tax_benefit_system)
        self.year = "2021-03"
        self.init_from_data(data = data)


In [89]:
qf = [410, 510, 620, 720, 820, 920, 1030, 1540, 2050, 2051, 2052]

In [90]:
counts = [4025, 2113, 1832, 1603, 962, 638, 566, 1990, 1157, 2090, 45]

In [93]:
input_data_frame = pd.DataFrame({
    'strasbourg_metropole_quotient_familial': qf,
    'famille_role_index': 0,
    'foyer_fiscal_role_index': 0,
    'menage_role_index': 0,
    'famille_id': range(len(qf)),
    'foyer_fiscal_id': range(len(qf)),
    'menage_id': range(len(qf)),
    'nombre_individus_inscrits': counts,
    'nombre_familles': counts,
    'nombre_foyers_fiscaux': 1,
    'nombre_menages': 1, 
    'strasbourg_metropole_nombre_repas_cantine': [200, 150, 200, 100, 145, 200, 150, 130, 180, 200, 150]
    })
input_data_frame

ValueError: arrays must all be same length

In [63]:
data = dict(input_data_frame = input_data_frame)
scenario = StrasbourgSurveyScenario(data = data)

# Tarif des cantines par familles

In [64]:
tarif_cantine = scenario.simulation.calculate('strasbourg_metropole_tarification_cantine', period = '2021-03')
tarif_cantine

array([1.5 , 2.15, 2.7 , 3.25, 3.8 , 4.4 , 5.1 , 5.8 , 6.3 , 6.9 , 6.9 ],
      dtype=float32)

# Recettes par tranches / types de familles

In [65]:
sum_tranches = counts * tarif_cantine * 200 # Pour 200 repas par an
sum_tranches

array([1207500.        ,  908590.04030228,  989280.01747131,
       1041950.        ,  731119.99082565,  561440.01216888,
        577319.98920441, 2308400.07591248, 1457820.04413605,
       2884200.03986359,   62100.00085831])

# Recettes totales

In [68]:
sum(sum_tranches)

12729720.21074295

In [69]:
scenario.compute_aggregate('strasbourg_metropole_cout_cantine', period = '2021-03', weighted= False)

7986.5

In [70]:
9760.0 / 200

48.8

In [71]:
scenario.weight_variable_by_entity = {'individu': 'nombre_individus_inscrits', 'famille': 'nombre_familles',
                                     'foyer_fiscal': 'nombre_foyers_fiscaux', 'menage': 'nombre_menages'}


In [72]:
scenario.compute_aggregate('strasbourg_metropole_cout_cantine', period = '2021-03')

10666962.5

In [73]:
counts 

[4025, 2113, 1832, 1603, 962, 638, 566, 1990, 1157, 2090, 45]

In [74]:
sum(counts)

17021